In [5]:
#!pip install streamlit-chat

In [6]:
#!pip install streamlit

In [7]:
#!pip install langchain

In [8]:
#!pip install faiss-cpu

In [11]:
import streamlit as st # 웹 인터페이스 구성
from streamlit_chat import message # 스트림릿에서 제공하는 대화형 메시지 인터페이스
from langchain.embeddings.openai import OpenAIEmbeddings  # OpenAI의 임베딩 모델을 사용하여 텍스트를 고차원 벡터로 변환 (* 문서 검색 및 비교에 사용)
from langchain.chat_models import ChatOpenAI  # Chat 모델을 사용하여 자연어 대화를 생성
from langchain.chains import ConversationalRetrievalChain  # 대화형 검색 체인 구축을 위해 사용, 사용자의 질문에 대해 관련 문서를 검색하고 답변을 생성
from langchain.vectorstores import FAISS  # 변환된 대규모 벡터를 효율적으로 검색
import tempfile  # 임시 파일을 생성하고 관리
from langchain.document_loaders import PyPDFLoader  # PDF 문서를 로드하여 텍스트로 변환

import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

uploaded_file = st.sidebar.file_uploader("upload", type="pdf")  # 스트림릿에서 사용자가 PDF 파일을 업로드할 수 있는 사이드바 파일 업로더를 생성

In [13]:

# 업로드된 파일 처리 및 초기 설정
if uploaded_file :
    # 파일 업로드 처리 - 사용자가 업로드한 PDF 파일을 임시 파일로 저장
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file: # 임시 파일(tempfile)을 생성
                                                                # (delete=False) = 파일이 닫힌 후에도 삭제되지 않도록 설정. 이로 인해 파일 경로를 나중에 참조
        tmp_file.write(uploaded_file.getvalue())  # PDF 파일의 내용을 임시 파일에 쓴다.
                                                  # 업로드된 PDF 파일의 내용을 임시 파일에 쓴다 = 이 과정은 업로드된 파일의 바이너리 데이터를 임시 파일에 복사
        tmp_file_path = tmp_file.name # 임시 파일의 경로를 반환
    
    loader = PyPDFLoader(tmp_file_path) # PDF 파일을 로드
    
    data = loader.load()
    # PDF 파일을 열고, 그 내용을 텍스트로 추출
    # 추출된 텍스트는 리스트 형식으로 반환
    # 각 항목은 PDF 파일의 각 페이지에 해당하는 텍스트 데이터를 포함

    embeddings = OpenAIEmbeddings()  # 추출된 텍스트 데이터를 임베딩(벡터)로 변환
    vectors = FAISS.from_documents(data, embeddings)  # FAISS를 사용하여 임베딩된 데이터를 벡터 스토어에 저장

    
    # 대화형 검색 체인 생성
    chain = ConversationalRetrievalChain.from_llm(llm = ChatOpenAI(temperature=0.0,model_name='gpt-4'), retriever=vectors.as_retriever())
    # OpenAI의 GPT-4 모델을 사용하여 질문에 대한 답변을 생성
    # FAISS 벡터 스토어에서 관련 문서를 검색

    # => 이 체인은 사용자의 질문을 받아서 관련 정보를 벡터 스토어(FAISS)에서 검색하고, 검색된 정보를 바탕으로 GPT-4 모델이 응답을 생성

    # 대화 함수
    def conversational_chat(query):  #문맥 유지를 위해 과거 대화 저장 이력에 대한 처리      
        result = chain({"question": query, "chat_history": st.session_state['history']}) # 사용자의 질문을 받아서 대화형 검색 체인에 전달
        # 답변을 st.session_state['history']에 저장하여 문맥을 유지
        st.session_state['history'].append((query, result["answer"]))        
        return result["answer"]

    # 세션 상태 초기화 = Streamlit의 세션 상태를 초기화하는 역할
    # st.session_state를 사용하여 대화 히스토리와 생성된 메시지를 저장, 처음 로드되었을 때 초기값을 설정
    # 사용자가 상호작용하는 동안은 상태를 유지

   
    # 각 세션 상태에 해당하는 키(history, generatied, past)가 없으면(즉, 애플리케이션이 처음 로드되었거나 세션이 초기화된 경우) 각 조건에 맞게 초기화
    if 'history' not in st.session_state: # 사용자가 입력한 질문과 챗봇의 응답을 저장
        st.session_state['history'] = [] 
    
    if 'generated' not in st.session_state: # 챗봇이 생성한 메시지를 저장
        st.session_state['generated'] = ["안녕하세요! " + uploaded_file.name + "에 관해 질문주세요."]

    if 'past' not in st.session_state: # 사용자가 입력한 모든 질문을 저장
        st.session_state['past'] = ["안녕하세요!"]
        

    # UI 컨테이너 설정 : Streamlit의 컨테이너를 사용하여 챗봇의 응답과 사용자의 입력을 별도로 관리

    #챗봇 이력에 대한 컨테이너
    response_container = st.container()
    #사용자가 입력한 문장에 대한 컨테이너
    container = st.container()

    # 사용자 입력 처리(사용자가 입력한 문장에 대한 컨테이너)
    with container: #대화 내용 저장(기억)
        # st.form을 사용하여 사용자의 입력을 받음.
        with st.form(key='Conv_Question', clear_on_submit=True): # clear_on_submit=True = 제출 후 입력 필드를 지우도록 설정 
            user_input = st.text_input("Query:", placeholder="PDF파일에 대해 얘기해볼까요? (:", key='input')
            submit_button = st.form_submit_button(label='Send') # 폼을 제출하는 버튼을 생성
        
        # 사용자 입력 처리(조건 : 제출 버튼이 눌리고, 입력 필드가 비어있지 않다면(유저의 질문이 있다면))
        if submit_button and user_input:
            
            # 사용자가 입력한 질문을 conversational_chat 함수에 전달하여 챗봇의 응답을 생성
            output = conversational_chat(user_input)

            # 사용자가 입력한 질문을 past 리스트에 추가. 대화 히스토리가 업데이트
            st.session_state['past'].append(user_input)

            # 챗봇의 응답을 generated 리스트에 추가. 챗봇이 생성한 응답도 대화 히스토리에 추가
            st.session_state['generated'].append(output)

    
    # 대화 기록 표시
    if st.session_state['generated']: # 대화 기록이 있는지 확인
        with response_container: # 응답을 표시할 컨테이너를 설정
            for i in range(len(st.session_state['generated'])): # 대화 기록을 한 번에 표시하기 위해 for문 루프 실행
                message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style = "fun-emoji", seed = "Nala") # past 에서 사용자가 입력한 메시지 가져와서 표시
                message(st.session_state["generated"][i], key=str(i), avatar_style = "bottts", seed = "Fluffy") # generated 리스트에서 챗봇이 생성한 응답을 가져와서 표시